In [1]:
import numpy as np 
import pandas as pd
import sklearn 
import tensorflow as tf 
import nltk
import gensim
import json

2023-11-04 03:25:48.889434: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Load Data

In [6]:
##TO-DO: Read dataset.json as a dictionary object 
with open('dataset.json', 'r') as file:
    json_str = file.read()
    json_dic = json.loads(json_str)

In [7]:
##TO-DO: Convert dictionary object to pandas dataframe 
df = pd.DataFrame.from_dict(json_dic).T
for i in range (len(df.index)):
    df['post_tokens'].iloc[i] = " ".join(df['post_tokens'].iloc[i])
df.head()

,post_id,annotators,rationales,post_tokens
1179055004553900032_twitter,1179055004553900032_twitter,"[{'label': 'normal', 'annotator_id': 1, 'targe...",[],i dont think im getting my baby them white 9 h...
1179063826874032128_twitter,1179063826874032128_twitter,"[{'label': 'normal', 'annotator_id': 1, 'targe...",[],we cannot continue calling ourselves feminists...
1178793830532956161_twitter,1178793830532956161_twitter,"[{'label': 'normal', 'annotator_id': 4, 'targe...",[],nawt yall niggers ignoring me
1179088797964763136_twitter,1179088797964763136_twitter,"[{'label': 'hatespeech', 'annotator_id': 1, 't...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",<user> i am bit confused coz chinese ppl can n...
1179085312976445440_twitter,1179085312976445440_twitter,"[{'label': 'hatespeech', 'annotator_id': 4, 't...","[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",this bitch in whataburger eating a burger with...


In [8]:
from nltk.tokenize import word_tokenize

nltk.download('punkt')

df['tokenized'] = df.apply(lambda row: nltk.word_tokenize(row['post_tokens']), axis=1)

print(df['tokenized'])




[nltk_data] Downloading package punkt to /Users/23brianc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


1179055004553900032_twitter    [i, dont, think, im, getting, my, baby, them, ...
1179063826874032128_twitter    [we, can, not, continue, calling, ourselves, f...
1178793830532956161_twitter                  [nawt, yall, niggers, ignoring, me]
1179088797964763136_twitter    [<, user, >, i, am, bit, confused, coz, chines...
1179085312976445440_twitter    [this, bitch, in, whataburger, eating, a, burg...
                                                     ...                        
9989999_gab                    [if, ur, still, on, twitter, tell, carlton, i,...
9990225_gab                    [when, i, first, got, on, here, and, said, i, ...
9991681_gab                    [was, macht, der, moslem, wenn, der, zion, geg...
9992513_gab                    [it, is, awful, look, at, world, demographics,...
9998729_gab                    [the, jewish, globalist, elite, have, only, im...
Name: tokenized, Length: 20148, dtype: object


### Pre-Process Data

In [10]:
from itertools import chain
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')

lemmatizer = nltk.stem.WordNetLemmatizer()

def clean_text(text_features):
    
    text_features = text_features.str.lower()

    stop_words = stopwords.words('english')
    
    # df['tweet_without_stopword'] = text_features.apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
    df['tweet_without_stopword'] = text_features.apply(lambda x:[word for word in x.split() if word not in (stop_words)])

    tweets_without_stopword = list(df['tweet_without_stopword'])

    lemmatized = []
    
    for words in tweets_without_stopword: #lemmatize each word in df['tweet_without_stopword'] and create new column
        temp = []

        for w in words:
            temp.append(lemmatizer.lemmatize(w))
            
        # lemmatized.append(' '.join(temp))
        lemmatized.append(temp)
        
    df['text_lemmatized'] = lemmatized
    
    """
    pre-process text features of our dataset:  
    - Case Lowering
    - Tokenization
    - Stop Word Removal
    - Lemmatization 
 
    Paramaters
    ---------
    text_features: text (tweet) data in our dataframe.
    """

    ##TO-DO: complete this function!
    ##When determining how to return the processed data, consider the format your data needs to be in for train_test_split and model training

 

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/23brianc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/23brianc/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
clean_text(df['post_tokens'])

df.head()

,post_id,annotators,rationales,post_tokens,tokenized,tweet_without_stopword,text_lemmatized
1179055004553900032_twitter,1179055004553900032_twitter,"[{'label': 'normal', 'annotator_id': 1, 'targe...",[],i dont think im getting my baby them white 9 h...,"[i, dont, think, im, getting, my, baby, them, ...","[dont, think, im, getting, baby, white, 9, two...","[dont, think, im, getting, baby, white, 9, two..."
1179063826874032128_twitter,1179063826874032128_twitter,"[{'label': 'normal', 'annotator_id': 1, 'targe...",[],we cannot continue calling ourselves feminists...,"[we, can, not, continue, calling, ourselves, f...","[cannot, continue, calling, feminists, rights,...","[cannot, continue, calling, feminist, right, w..."
1178793830532956161_twitter,1178793830532956161_twitter,"[{'label': 'normal', 'annotator_id': 4, 'targe...",[],nawt yall niggers ignoring me,"[nawt, yall, niggers, ignoring, me]","[nawt, yall, niggers, ignoring]","[nawt, yall, nigger, ignoring]"
1179088797964763136_twitter,1179088797964763136_twitter,"[{'label': 'hatespeech', 'annotator_id': 1, 't...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",<user> i am bit confused coz chinese ppl can n...,"[<, user, >, i, am, bit, confused, coz, chines...","[<user>, bit, confused, coz, chinese, ppl, acc...","[<user>, bit, confused, coz, chinese, ppl, acc..."
1179085312976445440_twitter,1179085312976445440_twitter,"[{'label': 'hatespeech', 'annotator_id': 4, 't...","[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",this bitch in whataburger eating a burger with...,"[this, bitch, in, whataburger, eating, a, burg...","[bitch, whataburger, eating, burger, top, bun,...","[bitch, whataburger, eating, burger, top, bun,..."


In [17]:
print(len(df))

20148


In [22]:
from gensim.models import Word2Vec

def flatten(l):
    return [item for sublist in l for item in sublist]

def vectorize_text(cleaned_text, use_word2vec=True):
    
    model = Word2Vec(cleaned_text, min_count=1)
    
    feature_matrix = []
    
    for sentence in cleaned_text:
        
        vectors = []
        num_words = len(sentence)
        
        for word in sentence:
            vector = model.wv[word].tolist()
            vectors.append(vector)
        
        averaged_vector = []
        
        for i in range (0, len(vectors[0])):
            sum = 0
            
            for v in vectors:
                sum = sum + v[i]
                
            avg = sum/num_words
            
            averaged_vector.append(avg)
            
        feature_matrix.append(averaged_vector)
        
    return feature_matrix


    """
    use Word2Vec or Sentence2Vec to vectorize the cleaned text data
 
    Paramaters
    ---------
    cleaned_text: (cleaned) text (tweet) data in our dataframe.
    """
    ##TO-DO: complete this function!
    ##When determining how to return the processed data, consider the format your data needs to be in for train_test_split and model training


In [25]:
clean_text(df['post_tokens'])

feature_matrix = vectorize_text(df['text_lemmatized'])

print(len(feature_matrix))
print (len(feature_matrix[3]))

20148
100


In [24]:
feature_matrix

[[-0.44303963304712224,
  0.5285526875119942,
  -0.08114388248381707,
  -0.2822440971548741,
  0.542418722063303,
  -0.9493793294979975,
  0.11578748930388919,
  1.565670220897748,
  -0.26656234816003305,
  -0.5945114246927775,
  -0.25833564423597777,
  -0.6783851024049979,
  -0.2269718853326944,
  0.13718660437287047,
  0.3169051225368793,
  -0.3054468199037589,
  0.32550654674951846,
  -0.7753581771483788,
  0.42244739543933135,
  -1.2758457666406264,
  0.35917208515680754,
  0.1165854730285131,
  0.7110747846846397,
  -0.2800712985201524,
  -0.027007676302813567,
  0.22776667214930058,
  -0.19986031555499023,
  -0.3190438041033653,
  -0.30203422603125757,
  0.3910939326653114,
  0.8441284671425819,
  0.10247094499377105,
  0.021982545964419842,
  -0.6342971700315292,
  -0.056709993608260095,
  0.8109029305095856,
  0.0915771606495792,
  -0.07992303736794454,
  -0.32989549522216505,
  -1.0585385967905705,
  0.37694173492491245,
  -0.8875293714495806,
  -0.11455217039642426,
  -0.0661

### Model Training: KNN/Random Forest/RNNs

In [ ]:
from sklearn.model_selection import train_test_split

## TO-DO: split our data into train and test data

##### Using KNN
###### Refer to this resource: https://scikit-learn.org/stable/modules/neighbors.html#nearest-neighbors-classification

In [ ]:
## TO-DO: Determine initial value of parameters (especially K)
k = 0

## TO-DO: determine the best distance computation metric
candidate_metrics = ['euclidean','cosine','manhattan']
best_metric = ''




In [ ]:
from sklearn.neighbors import KNeighborsClassifier

## TO-DO: fit a KNN Classifier with distance metric and K 

In [ ]:
# TO-DO: calculating accuracy
from sklearn.metrics import accuracy_score


In [ ]:
## TO-DO: optimize K with hyperparameter tuning

from sklearn.model_selection import cross_val_score


## TO-DO: Visualize results to determine best K


In [ ]:
## TO-DO: repeat training with optimized K

##### Using XGBoost/Random Forest
###### Refer to this resource: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [ ]:
## TO-DO: Train model naively, with default parameters
from sklearn.ensemble import RandomForestClassifier

In [ ]:
## TO-DO: Evaluate base model
from sklearn.metrics import accuracy_score



In [ ]:
##TO-DO: Hyperparameter Tuning via RandomizedSearch & GridSearch 

## get current parameters and their values 
from pprint import pprint


## Randomized Search 
from sklearn.model_selection import RandomizedSearchCV
random_grid = {}
rf_random = RandomizedSearchCV()

## Grid Search
from sklearn.model_selection import GridSearchCV
param_grid = {}
grid_search = GridSearchCV()

##### Using LSTMs
###### Refer to this resource: https://www.youtube.com/watch?v=8HyCNIVRbSU&ab_channel=TheA.I.Hacker-MichaelPhi

In [ ]:
from tensorflow.python.keras.layers import LSTM, Activation, Dropout, Dense, Embedding
from tensorflow.python.keras.models import Sequential
from keras_preprocessing.sequence import pad_sequences


In [ ]:
## veryyyyyy basic architecture. TO-DO: complete and expand

model = Sequential()
model.add(Embedding())
model.add(LSTM())
model.add(Dense(2, activation='softmax'))

model.compile(loss='',optimizer='', metrics=['accuracy'])

In [ ]:
epochs = 0
batch_size = 0

## TO-DO: determine training parameters and complete call
model.fit()

In [ ]:
## TO-DO: evaluate function 
model.evaluate()

### Using BERT
###### Refer to: http://jalammar.github.io/illustrated-bert/ 
###### Refer to: https://medium.com/@parthdholakiya180/twitter-hate-detection-using-bert-e7682b2d0a0c 

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer